# Phys 581 Winter 2019
# Final Project: Computational methods in quantum many-body physics
## Alexander Hickey, 10169582

Note that the contents of this notebook were created and tested in a 64-bit distribution of Windows 10, using Python 3.6.8.

In [1]:
import sys
sys.version

'3.6.8 |Anaconda, Inc.| (default, Feb 11 2019, 15:03:47) [MSC v.1915 64 bit (AMD64)]'

In [2]:
#Import useful libraries
import numpy as np
import scipy.optimize
import timeit
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction

Over the past two-decades, the increasing computational power available to researchers has lead to significant advances in the area of quantum many-body physics. Numerical methods continue to play an increasingly prominent role in this field. Despite these advances, simulating many-body systems remains a difficult task, as the underlying Hilbert space scales exponentially with the number of particles. To get around this, various analytical and numerical techniques have been explored to approximate solutions of these types of problems.

Recently, there has been immense experimental advances in the trapping of ultracold atoms in optical lattices (atomic lattices constructed by interfering lasers), which has renewed interest in lattice models of neutral bosonic atoms. Such environments serve as ideal playgrounds to study the dynamics of quantum many-body systems, as experimentalists have precise control of the underlying lattice potential, and the properties of ultracold atoms can be widely tuned with external fields. In particular, the versatility of optical lattice environments has lead to the realization of a wide range of quantum phase transitions, which refers to phase transitions that occur only at low temperatures, where thermal fluctuations are strongly suppressed. 

The canonical example of such a model is the Bose-Hubbard model, which describes an approximate Hamiltonian for atoms in an optical lattice, and consists of bosons hopping between discrete lattice sites, and interacting at close range. It has been shown that in such a model, the competing lattice tunnelling and interaction terms drive a quantum phase transition between superfluid and insulating phases. Since then, the Bose-Hubbard model has been realized experimentally, and extensions to the model have been of great interest from both the experiment and theory point of view.

The main difficulty that arises in the theoretical study of the Bose-Hubbard model is that the Hamiltonian cannot be diagonalized analytically whenever interactions are present. Additionally, numerical studies are restricted by the exponential scaling of the lattice. The latter problem role is generally supressed by a studying the model within a mean-field theory, where long range correlations are approximated to first order by replacing some operators by their average values.

This notebook presents the explorations of various computational methods that can be used to study the Bose-Hubbard model in the context of a zero-temperature mean-field theory. In particular, we will explore the method of exact diagonalization, variational methods, as well as imaginary-time propagation to compute ground states in the Bose-Hubbard model for various parameters. The benchmarks used to compare each method will be ability of the method to produce a mean-field phase diagram that is consistent with current literature, as well as the time required to perform such a task.



## Background

### Bosons

There are many systems in nature that are comprised of several identical particles. Such particles all share the same intrinsic properties, such as charge, mass and spin, and therefore cannot be distinguished by measuring their underlying characteristics. In classical mechanics, these particles can be readily distinguished by following their individual trajectories through phase space. In quantum mechanics however, such a trajectory is ill-defined due to the Heisenberg uncertainty principle.

This principle of indistinguishibility turns out to be quite fundamental, as it implies that many identical quantum particles obey either Fermi or Bose statistics. Particles that obey the former are known as *fermions*, and they are characterized by obeying the Pauli exclusion principle, which states that no two particles can occupy the same quantum state. In contrast, particles that obey Bose statistics are known as *bosons*, and an arbitrary number of them can occupy the same state. In this work, we will consider only bosonic particles, which is motivated by the fact that the aforementioned optical lattice experiments are often performed with neutral bosonic atoms, most notably with $^{87}\text{Rb}$, which was used to create the first Bose-Einstein condensate in 1995.

Since the particles are indistinguishable, we don't care about which particle is where, but rather just how many bosons are occupying each available quantum state. In our case, these so called states will correspond to a particles position on a 1D lattice. A convenient choice of an orthonormal basis for representing a many-body bosonic wavefunction is therefore the *occupation number basis*, where the basis vectors look like 

$$| n_0 ,n_1,n_2,\ldots \rangle$$

which corresponds to the state with $n_0$ bosons occupying the $0^\text{th}$ lattice site, $n_1$ bosons occupying the $1^\text{st}$ lattice site, and so on. Any arbitrary many-body wavefunction can then be written as a superposition of vectors in the occupation number basis. 

We would now like to be able to describe operators in this representation. To each lattice site, we associate a pair of operators $\hat a_j^\dagger$ and $\hat a_j$, known as creation and annihilation operators, which obey the commutator relations

$$[ \hat a_j ;  \hat a_k] = 0  \qquad [ \hat a_j, \hat a_k^\dagger] = \delta_{jk}$$

In particular, the action of these operators on an occupation number basis state is given by

\begin{align}
\hat a_j | n_0,n_1, \ldots,n_j ,\ldots \rangle &= \sqrt{n_j} | n_0,n_1, \ldots,n_j-1 ,\ldots \rangle \\
\hat a_j^\dagger | n_0,n_1, \ldots,n_j ,\ldots \rangle &= \sqrt{n_j+1} | n_0,n_1, \ldots,n_j +1,\ldots \rangle .
\end{align}

That is, the action of a creation (annihilation) operator is to quite literally create (annihilate) a boson that is occupying the $j^\text{th}$ state. It is also convenient to define a number operator

$$ \hat n_j  =  \hat a_j^\dagger \hat a_j $$

where the action of the number operator is to count the number of bosons occupying the $j^\text{th}$ lattice site, i.e.

$$\hat n_j | n_0,n_1, \ldots,n_j ,\ldots \rangle = n_j  | n_0,n_1, \ldots,n_j ,\ldots \rangle .$$

### The Bose-Hubbard model

Next, I will formally introduce the Bose-Hubbard model, which serves as an approximate model for bosons in an optical lattice environment.

An optical lattice is a periodic potential created by interfering laser light. If an atom occupies this lattice, it will have some probability of tunnelling to a different lattice site, with the probability depending on the geometry of the underlying potential. In this work, we will consider only bosonic atoms, which in principle is any atom with an even total number of constituent electrons, protons and neutrons. Such atoms obey bosonic statistics, meaning that an arbitrary number of atoms can share the same quantum state.

THe Bose-Hubbard model simplifies the dynamics of atoms confined to an optical lattice by discretizing each potential well to a specific lattice site, which either contains atoms or doesn't. The model describes these bosons tunnelling between nearest-neighbour lattice sites, as well as a repulsive interaction between bosons on the same lattice site. The Hamiltonian for this model is 

\begin{equation}  
\hat H = -t \sum_{\langle j,k \rangle} \hat a_{j}^\dagger \hat a_{k} + \frac U2 \sum_{j} \hat n_{j} \left( \hat n_{j} -1 \right) -\mu \sum_{j} \hat n_{j}
\end{equation}

where $\langle j,k \rangle$ indicates the sum be taken over nearest neighbours, $\hat a_j^\dagger$ and $\hat a_j$ are the bosonic creation and annihilation operators acting on the $j^\text{th}$ lattice site, and $\hat n_j = \hat a_j^\dagger \hat a_j$ is the number operator. In particular, this means that the action of the $\hat a_j^\dagger$ $\left( \hat a_j \right)$ is to create (remove) a boson from the $j^\text{th}$ site of the lattice, and the action of the $\hat n_j$ operator is to count the number of bosons on this site. The first term in the Hamiltonian corresponds to nearest-neighbour tunnelling between sites (see Fig. 1), and the energy $t$ is known as the \textit{tunnelling amplitude}, which is a measure of how likely a boson is to hop to a neighbouring site. This term favours the delocalization of bosons across the lattice whenever $t >0$. The second term models an on-site repulsive force, by counting the $\binom{n_j}{2} = \frac 12 n_j(n_j-1)$ mutual interactions of energy $U$ between bosons on the $j^\text{th}$ site. The last term in the Hamiltonian is the chemical potential, which fixes the average particle number in the grand canonical ensemble.



![Hopping on a lattice](lattice.png)

Fig. 1. Hopping on a 1D lattice. The operator $\hat a_{j-1}$ deletes a boson on site $j-1$, and the operator $\hat a_j^\dagger$ creates a boson on site $j$. The action of the operator $\hat a_j^\dagger \hat a_{j-1}$ therefore corrsponds to a boson hopping to the right, and its adjoint represents a boson hopping to the left. Figure made in TikZ.


### Mean field theory

As mentioned previously, the main problem that arises in numerical studies of the Bose-Hubbard model is the exponential scaling of the Hilbert space with the number of lattice sites. The typical way to make the problem more feasible for numerical simulations is to decouple the lattice sites from one another, by employing a mean field theory.

To do this, we will look specifically at the operators in the Hamiltonian that involve products of operators acting on different lattice sites. These operators take the form $\hat a_{j}^\dagger \hat a_{k}$ where $j,k$ correspond to indices of neighbouring lattice sites. We can define a set of "fluctuation operators"

\begin{equation}
\delta \hat a_{j} \equiv \hat a_{j} - \psi_j
\end{equation}

where $\psi_j \equiv \langle \hat a_{j} \rangle$ is the expectation value of $\hat a_j$ in the ground state. It follows that $\langle \delta \hat a_j \rangle \approx 0$ for states that are close to the true ground state. We can then rewrite the products of operators acting on different sites as:

\begin{align*}
\hat a_{j}^\dagger \hat a_{k} &= \Big( \delta \hat a_{j }^\dagger +\psi_j^* \Big)  \Big( \delta \hat a_{k } +\psi_k \Big)  \\
&= \delta \hat a_{j}^\dagger \delta \hat a_{k} +  \psi_j^*  \delta \hat a_{k } + \delta \hat a_{j}^\dagger \psi_k + \psi_{j}^* \psi_k . \\
\end{align*}

The mean field approximation is then to assume that correlations are small enough that we can ignore the term which is second order in the fluctuation operator. Under this assumption, the product becomes

\begin{align*} 
\hat a_{j}^\dagger \hat a_{k} &\approx \psi_j^*  \delta \hat a_{k } + \delta \hat a_{j}^\dagger \psi_k + \psi_{j}^* \psi_k \\
&=   \psi_j^*  \hat a_{k } +\hat a_{j}^\dagger \psi_k - \psi_{j}^* \psi_k 
\end{align*}

which is simply a sum of single site operators multiplied by a scalar expectation value. Next, we will assume that the lattice is effectively infinite in size, and is therefore translationally symmetric. Each lattice site will therefore look the same, and we can assume that $\psi_0 = \psi_1 = \cdots = \psi_N \equiv \psi$. With the aforementioned approximations, the Bose-Hubbard Hamiltonian becomes 

$$H = -t \sum_j  \left[ \psi^* \hat a_j + \psi \hat a_j^\dagger - |\psi|^2 \right] + \frac U2 \sum_{j} \hat n_{j} \left( \hat n_{j} -1 \right) -\mu \sum_{j} \hat n_{j}$$

This choice of mean field approximation effectively decouples operators acting on each site from one another, by replacing the operator with its average value. By decoupling the sites from one another, the system of interest will now scale linearly with the number of lattice sites, rather than exponentially, and is thus manageable for efficient numerical calculations.

The problem that remains is that we don't know the actual value of the mean-field parameter $\psi$. This can be obtained by assuming that the system is in equilibrium, and so the actual valus of $\psi$ will be the one that minimizes the free energy of the system. Furthermore, we are interested in the dynamics of this system at zero temperature, so the free energy corresponds to the total internal energy.

### Probing for phase transitions

The final question that remains is how we will actually probe for phase transitions in the model. Previous studies of this model have shown that the ground state exists in two distinct phases, namely, a Mott-insulator and a superfluid. It turns out that the expectation value $\psi = \langle a \rangle$ corresponds to the superfluid order parameter, meaning that an insulating phase corresponds to $\psi = 0$, and a superfluid phase corresponds to $\psi \neq 0$.

## Methods

### Exact diagonalization

The first method that will be explored is the method of exact diagonalization. We will start by explicitely constructing the Hamiltonian matrix, by computing the matrix elements of the Hamiltonian for a single site

$$\langle m | H | n \rangle = -t \psi^* \sqrt{n} \delta_{m,n-1} - t\psi \sqrt{n+1} \delta_{m,n+1} + \left(t |\psi|^2 + \frac{U}{2} n(n-1) - \mu n\right) \delta_{m,n}$$

for a given parameter set $(t,U,\mu)$. By dividing by the on-site interaction energy $U$, we make this Hamiltonian dimensionless, and effectively reduce the problem to two free parameters $\left( \frac{t}{U},\frac{\mu}{U} \right)$. Some arbitrary state is taken to be our initial guess, and the value of $\psi$ is computed accordingly. In principle, any number of bosons can occupy the lattice site, however, we truncate the Hilbert space to $N=10$ particles per site, so the wavefunction takes the form

$$| \phi_0 \rangle = \sum_{x=0}^N \beta_x | x \rangle $$

for some $\beta_0,\beta_1,\ldots,\beta_N$. The Hamiltonian matrix is then numerically diagonalized, and the state $| \phi_0 \rangle$ is then updated to be the lowest energy eigenvector of this Hamiltonian. The Hamiltonian matrix elements are then recomputed using the new state, and the process is repeated until a self-consistent value of the order parameter $\psi$ is obtained. The phase of the ground state is then inferred using the value of $\psi$. 

### Variational principle

Check that results are consistent with the ansatz for $\psi$. i.e. let 
$$| \phi_0 \rangle = \sum_{x=0}^N \beta_x | x \rangle.$$ Then $$\psi = \langle \phi_0 | a^\dagger| \phi_0 \rangle = \sum_{x=0}^{N-1} \sqrt{x+1} \beta_x \beta_{x+1}$$ and $$\frac{E_0}{U} = \sum_{x=0}^N \left[\beta_x^2 \left( \frac 12 x(x-1) - \frac{\mu}{U} x \right) \right] - \frac{t}{U}\psi^2$$

### Imaginary-time propagation

### Conclusion

### Bibliography